__Dataloder FMD_to_Hagen_Transfer and Baseline_Hagen__

In [31]:
from fastai.vision.all import *
import cv2
import fastai; fastai.__version__

'2.7.15'

In [ ]:
def open_img(path):
    img_array = cv2.imread(str(path),-1)
    img_array = np.stack([img_array, img_array, img_array],0)
    img_array = img_array/65535
    img_array = img_array.astype('float32')
    img_tensor = torch.tensor(img_array, dtype=torch.float32)
    img_tensor = resize_tensor(img_tensor)
    return img_tensor

# resize the dimensions of a tensor, since TransformBlock has no "Resize" method
def resize_tensor(input_tensor):
    tensor_un = input_tensor.unsqueeze(0)
    tensor_res = torch.nn.functional.interpolate(tensor_un,size=(256,256), mode='bilinear', align_corners=True)
    tensor_sq = tensor_res.squeeze(0)
    return(tensor_sq)

In [2]:
# set up data loader via DataBlock API
set_seed(42)
def get_dls(bs:int):
    dblock = DataBlock(blocks=(TransformBlock(open_img), TransformBlock(open_img)),           # replace ImageBlock with TransformBlock
                       get_items=get_image_files,
                       get_y = lambda x: path_GT/x.name,
                       splitter=RandomSplitter(valid_pct=0.2),
                       batch_tfms=[Normalize.from_stats(*imagenet_stats)]
                                  )                                   
    dls = dblock.dataloaders(path_noisy, bs=bs, path=path)
    dls.c = 3
    return dls, dblock